In [27]:
import torch # torch will allow us to create tensors.
import torch.nn as nn # torch.nn allows us to create a neural network.
import torch.nn.functional as F # nn.functional give us access to the activation and loss functions.
from torch.optim import Adam # optim contains many optimizers. This time we're using Adam

import lightning as L # lightning has tons of cool tools that make neural networks easier
from torch.utils.data import TensorDataset, DataLoader # these are needed for the training data

import pandas as pd # We'll use pandas to read in the data and normalize it
from sklearn.model_selection import train_test_split # We'll use this to create training and testing datasets


In [28]:
df = pd.read_table("./iris.txt", sep=",", header=None)
df

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [29]:
## To name each column, we assign a list of column names to `columns`
df.columns = ["sepal_length", "sepal_width", "petal_length", "petal_width", "class"]
df

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [30]:
df['class'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [31]:
for class_name in df["class"].unique():  # for each unique class name...

    ## ...print out the number of rows associated with it
    print(class_name, ": ", sum(df["class"] == class_name), sep="")

Iris-setosa: 50
Iris-versicolor: 50
Iris-virginica: 50


In [32]:
df['class'].value_counts()

class
Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: count, dtype: int64

In [33]:
input_values = df[["petal_width", "sepal_width"]]
label_values = df["class"]


In [34]:
classes_as_numbers = label_values.factorize()[0]


In [35]:
input_train,input_test,label_train, label_test=train_test_split(input_values,classes_as_numbers,test_size=0.3,stratify=classes_as_numbers)

In [36]:
input_train.shape

(105, 2)

In [37]:
label_train.shape

(105,)

In [38]:
one_hot_label_train = F.one_hot(torch.tensor(label_train)).type(torch.float32)
one_hot_label_train.shape

torch.Size([105, 3])

In [39]:
max_vals_in_input_train = input_train.max()
min_vals_in_input_train = input_train.min()

In [40]:
input_train=(input_train-min_vals_in_input_train)/(max_vals_in_input_train-min_vals_in_input_train)
input_train.head

<bound method NDFrame.head of      petal_width  sepal_width
75      0.541667     0.363636
120     0.916667     0.454545
8       0.041667     0.318182
77      0.666667     0.363636
83      0.625000     0.227273
..           ...          ...
74      0.500000     0.318182
20      0.041667     0.545455
9       0.000000     0.409091
148     0.916667     0.545455
81      0.375000     0.090909

[105 rows x 2 columns]>

In [41]:
input_test = (input_test - min_vals_in_input_train) / (
    max_vals_in_input_train - min_vals_in_input_train
)

In [42]:
input_train_tensors = torch.tensor(input_train.values).type(torch.float32)
input_test_tensors = torch.tensor(input_test.values).type(torch.float32)

In [43]:
train_dataset = TensorDataset(input_train_tensors, one_hot_label_train)
train_dataloader = DataLoader(train_dataset)

In [44]:
class MultipleInOuts(L.LightningModule):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        L.seed_everything(12)
        self.input_to_hidden=nn.Linear(in_features=2,out_features=2,bias=True)
        self.hidden_units=nn.Linear(in_features=2, out_features=3,bias=True)
        self.loss=nn.MSELoss(reduction='sum')
    def forward(self, inputs):
        hidden=self.input_to_hidden(inputs)
        output_values = self.hidden_units(torch.relu(hidden))
        return output_values

    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.001)

    def training_step(self,batch,batch_idx):
        input,labels=batch
        outputs=self.forward(inputs=input)
        loss=self.loss(outputs,labels)
        return loss

In [45]:
model = MultipleInOuts()

Seed set to 12


In [21]:
for name,param in nn.named_parameters():
    print(name, torch.round(param.data,decimals=2))

input_to_hidden.weight tensor([[-0.0500, -0.3800],
        [-0.0700,  0.1200]])
input_to_hidden.bias tensor([-0.1300, -0.5300])
hidden_units.weight tensor([[ 0.1900, -0.3600],
        [ 0.3300,  0.3100],
        [-0.4300,  0.2800]])
hidden_units.bias tensor([0.1200, 0.1900, 0.0800])


In [ ]:

trainer=L.Trainer(max_epochs=3)
trainer.fit(model,train_dataloaders=train_dataloader)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [48]:
%%capture
# %%capture prevents this cell from printing a ton of STDERR stuff to the screen

## First, check to see if lightning is installed, if not, install it.
##
## NOTE: If you **do** need to install something, just know that you may need to
##       restart your session for python to find the new module(s).
##
##       To restart your session:
##       - In Google Colab, click on the "Runtime" menu and select
##         "Restart Session" from the pulldown menu
##       - In a local jupyter notebook, click on the "Kernel" menu and select
##         "Restart Kernel" from the pulldown menu
import pip
try:
  __import__("lightning")
except ImportError:
  pip.main(['install', "lightning"])

In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import lightning as L
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split

In [50]:
url = "https://raw.githubusercontent.com/StatQuest/signa/main/chapter_04/iris.txt"
df = pd.read_table(url, sep=",", header=None)

In [51]:
df.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [52]:
## To name each column, we assign a list of column names to `columns`
df.columns = ["sepal_length", "sepal_width", "petal_length", "petal_width", "class"]

## To verify we did that correctly, let's print out the first few rows
df.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [53]:
df.shape

(150, 5)

In [54]:
df["class"].nunique()

3

In [55]:
## We can print out the unique values in a dataframe's column with the 'unique()' method.
df["class"].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [56]:
input_values = df[["petal_width", "sepal_width"]]
label_values = df["class"]

In [57]:
## Convert the strings in the 'class' column into numbers with factorize()
classes_as_numbers = label_values.factorize()[
    0
]  ## NOTE: factorize() returns a list of lists,
## and since we only need the first list of values,
## we index the output of factorize() with [0].
classes_as_numbers  ## print out the numbers

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [58]:
input_train, input_test, label_train, label_test = train_test_split(
    input_values, classes_as_numbers, test_size=0.25, stratify=classes_as_numbers
)

In [59]:
one_hot_label_train = F.one_hot(torch.tensor(label_train)).type(torch.float32)

In [61]:
type(one_hot_label_train)

torch.Tensor

In [62]:
## First, determine the maximum values in input_train...
max_vals_in_input_train = input_train.max()
## Now print them out...
max_vals_in_input_train

petal_width    2.5
sepal_width    4.4
dtype: float64

In [63]:
## Second, determine the minimum values in input_train
min_vals_in_input_train = input_train.min()
## Now print them out...
min_vals_in_input_train

petal_width    0.1
sepal_width    2.2
dtype: float64

In [64]:
## Now normalize input_train with the maximum and minimum values from input_train
input_train = (input_train - min_vals_in_input_train) / (
    max_vals_in_input_train - min_vals_in_input_train
)
input_train.head()

,petal_width,sepal_width
113,0.791667,0.136364
105,0.833333,0.363636
88,0.500000,0.363636
120,0.916667,0.454545
145,0.916667,0.363636


In [65]:
## Now normalize input_test with the maximum and minimum values from input_train
input_test = (input_test - min_vals_in_input_train) / (
    max_vals_in_input_train - min_vals_in_input_train
)
input_test.head()

,petal_width,sepal_width
98,0.416667,0.136364
108,0.708333,0.136364
126,0.708333,0.272727
13,0.000000,0.363636
142,0.750000,0.227273


In [66]:
## Convert the DataFrame input_train into tensors
input_train_tensors = torch.tensor(input_train.values).type(torch.float32)

## now print out the first 5 rows to make sure they are what we expect.
input_train_tensors[:5]

tensor([[0.7917, 0.1364],
        [0.8333, 0.3636],
        [0.5000, 0.3636],
        [0.9167, 0.4545],
        [0.9167, 0.3636]])

In [67]:
train_dataset = TensorDataset(input_train_tensors, one_hot_label_train)
train_dataloader = DataLoader(train_dataset)

In [68]:
class MultipleInsOuts(L.LightningModule):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.input_to_hidden=nn.Linear(in_features=2,out_features=2,bias=True)
        self.hidden_to_out=nn.Linear(in_features=2,out_features=3,bias=True)
        self.loss=nn.CrossEntropyLoss()

    def forward(self,input):
        hidden=self.input_to_hidden(input)
        output = self.hidden_to_out(torch.relu(hidden))
        return output
    def configure_optimizers(self):
        return Adam(self.parameters(),lr=0.001)
    def training_step(self, batch,batch_idx):
        inputs,labels=batch
        outputs=self.forward(inputs)
        loss=self.loss(outputs,labels)
        return loss
    

In [69]:
model=MultipleInOuts()

Seed set to 12


In [ ]:
trainer=L.Trainer(max_epochs=10)
trainer.fit(model,train_dataloader)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [ ]:
# Run the input_test_tensors through the neural network
predictions = model(input_test_tensors)
## Select the output with highest value...
predicted_labels = torch.argmax(
    predictions, dim=1
)  ## dim=0 applies argmax to rows, dim=1 applies argmax to columns
predicted_labels[0:4]  # print out the first 4 predictions

In [ ]:
torch.sum(torch.eq(torch.tensor(label_test), predicted_labels)) / len(predicted_labels)

In [ ]:
path_to_checkpoint = (
    trainer.checkpoint_callback.best_model_path
)  ## By default, "best" = "most recent"

In [ ]:
## First, create a new Lightning Trainer
trainer = L.Trainer(
    max_epochs=100
)  # Before, max_epochs=10, so, by setting it to 100, we're adding 90 more.

## Then call trainer.fit() using the path to the most recent checkpoint files
## so that we can pick up where we left off.
trainer.fit(model, train_dataloaders=train_dataloader, ckpt_path=path_to_checkpoint)